In [ ]:
# default_exp parser

# Trove query parser

> Use `parse_query` to convert a search query from the Trove web interface into a set of parameters that the API will understand.

In [ ]:
#hide
from nbdev.showdoc import *

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Functions

In [ ]:
#export

import os
from urllib.parse import urlparse, parse_qsl, parse_qs
import requests
import arrow

def format_date(date, start=False):
    '''
    The web interface uses YYYY-MM-DD dates, but the API expects YYYY-MM-DDT00:00:00Z. Reformat dates accordingly.
    
    Also the start date in an API query needs to be set to the day before you want. So if this is a start date, take it back in time by a day.
    '''
    if date != '*':
        date_obj = arrow.get(date)
        if start:
            date_obj = date_obj.shift(days=-1)
        date = '{}Z'.format(date_obj.format('YYYY-MM-DDT00:00:00'))
    return date

def parse_query(query, api_version=2):
    '''
    Converts the parameters of a search using the Trove web interface into a form the API will understand.
    
    Parameters:  
    * `query` – the url of a search in the Trove newspapers & gazettes category
    * `api_version` – Trove API version (default is 2)
    
    Returns:  
    * a dict containing the parameters (multiple values will be in a list)
    '''
    parsed_url = urlparse(query)
    if 'api.trove.nla.gov.au' in query:
        # If it's an API url, no further processing of parameters needed
        new_params = parse_qs(parsed_url.query)
    else:
        # These params can be accepted as is.
        safe = ['l-category', 'l-title', 'l-decade', 'l-year', 'l-month', 'l-state', 'l-word', 'include']
        new_params = {}
        dates = {}
        keywords = []
        params = parse_qsl(parsed_url.query)
        # Loop through all the parameters
        for key, value in params:
            if key in safe:
                try:
                    new_params[key].append(value)
                except KeyError:
                    new_params[key] = [value]
            elif key == 'l-advWord':
                new_params['l-word'] = value
            elif key == 'l-advstate':
                try:
                    new_params['l-state'].append(value)
                except KeyError:
                    new_params['l-state'] = [value]
            elif key == 'l-advcategory':
                try:
                    new_params['l-category'].append(value)
                except KeyError:
                    new_params['l-category'] = [value]
            elif key == 'l-advtitle':
                try:
                    new_params['l-title'].append(value)
                except KeyError:
                    new_params['l-title'] = [value]
            elif key in ['l-illustrationType', 'l-advIllustrationType']:
                new_params['l-illustrated'] = 'true'
                try:
                    new_params['l-illtype'].append(value)
                except KeyError:
                    new_params['l-illtype'] = [value]
            elif key == 'date.from':
                dates['from'] = value
            elif key == 'date.to':
                dates['to'] = value
            elif key == 'keyword':
                new_params['q'] = value
            elif key == 'keyword.phrase':
                keywords.append('"{}"'.format(value))
            elif key == 'keyword.not':
                keywords.append('NOT ({})'.format(' OR '.join(value.split())))
            elif key == 'keyword.any':
                keywords.append('({})'.format(' OR '.join(value.split())))
            elif key in ['l-ArtType', 'l-advArtType', 'l-artType']:
                if api_version == 2:
                    if value == 'newspapers':
                        new_params['zone'] = 'newspaper'
                    elif value == 'gazette':
                        new_params['zone'] = 'gazette'
                elif api_version == 3:
                    new_params['l-artType'] = value
        if keywords:
            if 'q' in new_params:
                new_params['q'] += ' AND {}'.format(' AND '.join(keywords))
            else:
                new_params['q'] = ' AND '.join(keywords)
        if dates:
            if 'from' not in dates:
                dates['from'] = '*'
            if 'to' not in dates:
                dates['to'] = '*'
            date_query = 'date:[{} TO {}]'.format(format_date(dates['from'], True), format_date(dates['to']))
            if 'q' in new_params:
                new_params['q'] += ' {}'.format(date_query)
            else:
                new_params['q'] = date_query
        if 'q' not in new_params:
            new_params['q'] = ' '
        if api_version == 2 and 'zone' not in new_params:
            new_params['zone'] = 'newspaper,gazette'
        if api_version == 3 and 'category' not in new_params:
            new_params['category'] = 'newspaper'
    # return '{}?{}'.format('https://api.trove.nla.gov.au/v2/result', urlencode(new_params, doseq=True))
    return new_params

In [ ]:
show_doc(parse_query)

<h4 id="parse_query" class="doc_header"><code>parse_query</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>parse_query</code>(**`query`**, **`api_version`**=*`2`*)

Converts the parameters of a search using the Trove web interface into a form the API will understand.

Parameters:  
* `query` – the url of a search in the Trove newspapers & gazettes category
* `api_version` – Trove API version (default is 2)

Returns:  
* a dict containing the parameters (multiple values will be in a list)

## Basic usage

Here's the url of a search in Trove's newspapers: https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-state=Queensland&l-category=Article&l-illustrationType=Cartoon

If we feed this url to `parse_query()` we get back a dict with the query parameters translated into a form the Trove API understands.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-state=Queensland&l-category=Article&l-illustrationType=Cartoon', 3)
params

{'q': 'wragge',
 'l-artType': 'newspapers',
 'l-state': ['Queensland'],
 'l-category': ['Article'],
 'l-illustrated': 'true',
 'l-illtype': ['Cartoon'],
 'category': 'newspaper'}

If you want to use this to get data back from the Trove API, you'll need to provide your Trove API key, either as a query parameter (version 2), or in the request headers (version 3). You might also want to change the `encoding` of the results to 'json'. Then you can just give the parameters as `params` to `requests`. For example:

``` python
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-state=Queensland&l-category=Article&l-illustrationType=Cartoon', 3)
headers = {'X-API-KEY': 'mYApiKEY'}
params['encoding'] = 'json'
params['n'] = 1
response = requests.get('https://api.trove.nla.gov.au/v3/result', params=params, headers=headers)
data = response.json()
```

Assuming your API key is valid, this will return the following results:

``` python
{'query': 'wragge',
 'category': [{'code': 'newspaper',
   'name': 'Newspapers & Gazettes',
   'records': {'s': '*',
    'n': 2,
    'total': 510,
    'next': 'https://api.trove.nla.gov.au/v3/result?q=wragge&l-artType=newspapers&l-state=Queensland&l-category=Article&l-illustrated=true&l-illtype=Cartoon&category=newspaper&encoding=json&n=2&s=AoIIQzWFoig4MjM0NjM1NA%3D%3D',
    'nextStart': 'AoIIQzWFoig4MjM0NjM1NA==',
    'article': [{'id': '21765046',
      'url': 'https://api.trove.nla.gov.au/v3/newspaper/21765046',
      'heading': 'Mrs. Adelaide Wragge.',
      'category': 'Article',
      'title': {'id': '16',
       'title': 'The Brisbane Courier (Qld. : 1864 - 1933)'},
      'date': '1931-12-16',
      'page': '13',
      'pageSequence': '13',
      'relevance': {'score': 215.65185546875, 'value': 'very relevant'},
      'snippet': 'Formerly of Victoria, and in 1864 Mayoress of Melbourne, the late Mrs. Wragge, who died recently, had been',
      'troveUrl': 'https://.nla.gov.au/nla.news-article21765046?searchTerm=wragge'},
     {'id': '82346354',
      'url': 'https://api.trove.nla.gov.au/v3/newspaper/82346354',
      'heading': 'MR WRAGGE ON WEATHER CANNONS.',
      'category': 'Article',
      'title': {'id': '269',
       'title': 'The North Queensland Register (Townsville, Qld. : 1892 - 1905)'},
      'date': '1901-03-11',
      'page': '10',
      'pageSequence': '10',
      'relevance': {'score': 181.52200317382812, 'value': 'very relevant'},
      'snippet': 'I have been to Styria, have seen the cannons made in the forges, have witnessed the experiments, have visited Herr Stiger, the inventor of the',
      'troveUrl': 'https://.nla.gov.au/nla.news-article82346354?searchTerm=wragge'}]}}]}
```

Note that the API includes some additional parameters such as `reclevel` and `include`. Have a look at the [Trove API Console](https://troveconsole.herokuapp.com/) for examples.

## Version 2 tests

### Simple search with facets

Multiple keywords are just passed along as is and are combined with a boolean `AND`. This is the same in both the Simple and Advanced search.

In [ ]:
assert {'q': 'wragge weather', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20weather')

Multiple keywords with `OR` are passed along as is.

In [ ]:
assert {'q': 'wragge OR weather', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20OR%20weather')

Phrase search passed along as is.

In [ ]:
assert {'q': '"inclement wragge"', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=%22inclement%20wragge%22')

More complex queries such as date ranges should be passed along as is.

In [ ]:
parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20date%3A%5B1901%20TO%201903%5D&l-artType=newspapers')

{'q': 'wragge date:[1901 TO 1903]', 'zone': 'newspaper'}

Limit to gazettes using facets.

In [ ]:
assert {'q': 'wragge', 'zone': 'gazette'} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=gazette')

Limit state to NSW using facets.

In [ ]:
assert {'q': 'wragge', 'l-state': ['New South Wales'], 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-state=New%20South%20Wales')

Limit newspaper to SMH using facets.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-title': ['35']} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-title=35')

Limit to 'Article' category using facets.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-category': ['Article']} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-category=Article')

Limit to specific decade using facets.

In [ ]:
parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-decade=190')

{'q': 'wragge', 'zone': 'newspaper', 'l-decade': ['190']}

Limit to specific year using facets.

In [ ]:
parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-decade=190&l-year=1903')

{'q': 'wragge', 'zone': 'newspaper', 'l-decade': ['190'], 'l-year': ['1903']}

Limit to articles with illustration type of 'Photo' with facets.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-illustrated': 'true', 'l-illtype': ['Photo']} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-illustrationType=Photo')

Limit to articles containing more than 1,000 words using facets.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-word': ['1000+ Words']} == parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-word=1000%2B%20Words')

### Advanced search

Multiple keywords in 'Any of these words' box.

In [ ]:
assert {'q': '(wragge OR weather)', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.any=wragge%20weather')

Multiple keywords in 'The phrase' box.

In [ ]:
assert {'q': '"inclement wragge"', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.phrase=inclement%20wragge')

Keywords in 'All of these words' and 'Without these words' boxes.

In [ ]:
assert {'q': 'wragge AND NOT (weather)', 'zone': 'newspaper,gazette'} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.not=weather&keyword=wragge')

Limit to a specific date range.

In [ ]:
assert {'q': 'wragge date:[1899-12-31T00:00:00Z TO 1900-02-04T00:00:00Z]', 'zone': 'newspaper'} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&date.from=1900-01-01&date.to=1900-02-04&l-advArtType=newspapers')

Limit to a specific state.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-state': ['Queensland']} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advstate=Queensland')

Limit to specific newspapers.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-title': ['16', '1055']} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advtitle=16&l-advtitle=1055')

Limit to a specific category.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-category': ['Family Notices']} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advcategory=Family%20Notices')

Limit to a specific illustration type.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-illustrated': 'true', 'l-illtype': ['Photo']} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advIllustrationType=Photo')

Limit to a specific number of words.

In [ ]:
assert {'q': 'wragge', 'zone': 'newspaper', 'l-word': '100 - 1000 Words'} == parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advWord=100%20-%201000%20Words')

## Version 3 tests

### Simple search with facets

In [ ]:
def query_api(params):
    api_key = os.getenv("TROVE_API_KEY")
    params["n"] = 0
    response = requests.get("https://api.trove.nla.gov.au/v3/result", params=params, headers={"X-API-KEY": api_key})
    return response.status_code

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20weather', 3)
assert {'q': 'wragge weather', 'category': 'newspaper'} == params
assert query_api(params) == 200

Multiple keywords with `OR` are passed along as is.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20OR%20weather', 3)
assert {'q': 'wragge OR weather', 'category': 'newspaper'} == params
assert query_api(params) == 200

Phrase search passed along as is.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=%22inclement%20wragge%22', 3)
assert {'q': '"inclement wragge"', 'category': 'newspaper'} == params
assert query_api(params) == 200

More complex queries such as date ranges should be passed along as is.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge%20date%3A%5B1901%20TO%201903%5D&l-artType=newspapers', 3)
assert {'q': 'wragge date:[1901 TO 1903]', 'category': 'newspaper', 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to gazettes using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=gazette', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-artType': 'gazette'} == params
assert query_api(params) == 200

Limit state to NSW using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-state=New%20South%20Wales', 3)
assert {'q': 'wragge', 'l-state': ['New South Wales'], 'category': 'newspaper'} == params
assert query_api(params) == 200

Limit newspaper to SMH using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-title=35', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-title': ['35'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to 'Article' category using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-category=Article', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-category': ['Article'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to specific decade using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-decade=190', 3)
assert {'q': 'wragge', 'l-artType': 'newspapers', 'l-decade': ['190'], 'category': 'newspaper'} == params
assert query_api(params) == 200

Limit to specific year using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-decade=190&l-year=1903', 3)
assert {'q': 'wragge', 'l-artType': 'newspapers', 'l-decade': ['190'], 'l-year': ['1903'], 'category': 'newspaper'} == params
assert query_api(params) == 200

Limit to articles with illustration type of 'Photo' with facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-illustrationType=Photo', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-illustrated': 'true', 'l-illtype': ['Photo'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to articles containing more than 1,000 words using facets.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/category/newspapers?keyword=wragge&l-artType=newspapers&l-word=1000%2B%20Words', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-word': ['1000+ Words'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

### Advanced search

Multiple keywords in 'Any of these words' box.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.any=wragge%20weather', 3)
assert {'q': '(wragge OR weather)', 'category': 'newspaper'} == params
assert query_api(params) == 200

Multiple keywords in 'The phrase' box.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.phrase=inclement%20wragge', 3)
assert {'q': '"inclement wragge"', 'category': 'newspaper'} == params
assert query_api(params) == 200

Keywords in 'All of these words' and 'Without these words' boxes.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword.not=weather&keyword=wragge', 3)
assert {'q': 'wragge AND NOT (weather)', 'category': 'newspaper'} == params
assert query_api(params) == 200

Limit to a specific date range.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&date.from=1900-01-01&date.to=1900-02-04&l-advArtType=newspapers', 3)
assert {'q': 'wragge date:[1899-12-31T00:00:00Z TO 1900-02-04T00:00:00Z]', 'category': 'newspaper', 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to a specific state.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advstate=Queensland', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-state': ['Queensland'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to specific newspapers.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advtitle=16&l-advtitle=1055', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-title': ['16', '1055'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to a specific category.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advcategory=Family%20Notices', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-category': ['Family Notices'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to a specific illustration type.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advIllustrationType=Photo', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-illustrated': 'true', 'l-illtype': ['Photo'], 'l-artType': 'newspapers'} == params
assert query_api(params) == 200

Limit to a specific number of words.

In [ ]:
params = parse_query('https://trove.nla.gov.au/search/advanced/category/newspapers?keyword=wragge&l-advArtType=newspapers&l-advWord=100%20-%201000%20Words', 3)
assert {'q': 'wragge', 'category': 'newspaper', 'l-word': '100 - 1000 Words', 'l-artType': 'newspapers'} == params
assert query_api(params) == 200